# download example

In [ ]:
!wget https://raw.githubusercontent.com/Biuni/PokemonGO-Pokedex/master/pokedex.json

# write documents to sqlite db

In [1]:
import json
from SQLiteJSON import SQLiteJSON

In [2]:
with open("pokedex.json") as f:
    data = json.load(f)

In [3]:
len(data["pokemon"])

151

In [4]:
db = SQLiteJSON("pokedex.db")

In [5]:
db.write_json(data["pokemon"])

  0%|          | 0/151 [00:00<?, ?it/s]

In [6]:
db.close()

# read db

In [7]:
db = SQLiteJSON("pokedex.db")

In [8]:
db.TABLE

'docs'

In [9]:
db.BODY

'body'

In [10]:
db.size()

151

In [11]:
db.query("""PRAGMA table_list""")

[('schema', 'name', 'type', 'ncol', 'wr', 'strict'),
 ('main', 'sqlite_sequence', 'table', 2, 0, 0),
 ('main', 'docs', 'table', 2, 0, 0),
 ('main', 'sqlite_schema', 'table', 5, 0, 0),
 ('temp', 'sqlite_temp_schema', 'table', 5, 0, 0)]

In [12]:
db.query("""PRAGMA table_info('docs')""")

[('cid', 'name', 'type', 'notnull', 'dflt_value', 'pk'),
 (0, 'id', 'INTEGER', 0, None, 1),
 (1, 'body', 'JSON', 0, None, 0)]

In [13]:
db.query("""select count(body) from docs""")

[('count(body)',), (151,)]

# do some simple queries

query a document

In [14]:
db.query("select body from docs limit 1")

[('body',),
 ('{"id": 1, "num": "001", "name": "Bulbasaur", "img": "http://www.serebii.net/pokemongo/pokemon/001.png", "type": ["Grass", "Poison"], "height": "0.71 m", "weight": "6.9 kg", "candy": "Bulbasaur Candy", "candy_count": 25, "egg": "2 km", "spawn_chance": 0.69, "avg_spawns": 69, "spawn_time": "20:00", "multipliers": [1.58], "weaknesses": ["Fire", "Ice", "Flying", "Psychic"], "next_evolution": [{"num": "002", "name": "Ivysaur"}, {"num": "003", "name": "Venusaur"}]}',)]

query by key

In [15]:
db.query("""select body->>'$.name' from docs limit 10""")

[("body->>'$.name'",),
 ('Bulbasaur',),
 ('Ivysaur',),
 ('Venusaur',),
 ('Charmander',),
 ('Charmeleon',),
 ('Charizard',),
 ('Squirtle',),
 ('Wartortle',),
 ('Blastoise',),
 ('Caterpie',)]

filter values

In [16]:
db.query("""select body->>'$.name', body->>'$.spawn_chance' from docs where body->>'$.spawn_chance' < 0.01""")

[("body->>'$.name'", "body->>'$.spawn_chance'"),
 ('Charizard', 0.0031),
 ('Blastoise', 0.0067),
 ('Raichu', 0.0076),
 ('Ninetales', 0.0077),
 ('Vileplume', 0.0097),
 ('Alakazam', 0.0073),
 ('Machamp', 0.0068),
 ('Victreebel', 0.0059),
 ('Golem', 0.0047),
 ('Muk', 0.0031),
 ('Gengar', 0.0067),
 ('Kangaskhan', 0.0086),
 ('Mr. Mime', 0.0031),
 ('Gyarados', 0.0032),
 ('Lapras', 0.006),
 ('Ditto', 0),
 ('Omastar', 0.0061),
 ('Kabutops', 0.0032),
 ('Articuno', 0),
 ('Zapdos', 0),
 ('Moltres', 0),
 ('Dragonite', 0.0011),
 ('Mewtwo', 0),
 ('Mew', 0)]

count values in an array `type`

In [17]:
db.query("""select k.value, count(k.value) from docs, json_each(body->>'$.type') k group by k.value order by count(k.value)""")

[('value', 'count(k.value)'),
 ('Dragon', 3),
 ('Ghost', 3),
 ('Ice', 5),
 ('Fighting', 8),
 ('Electric', 9),
 ('Rock', 11),
 ('Bug', 12),
 ('Fire', 12),
 ('Grass', 14),
 ('Ground', 14),
 ('Psychic', 14),
 ('Flying', 19),
 ('Normal', 24),
 ('Water', 32),
 ('Poison', 33)]

filter if array contains a specific value

In [18]:
db.query("""select body from docs, json_each(body->>'$.type') k where k.value = 'Ice'""")

[('body',),
 ('{"id": 87, "num": "087", "name": "Dewgong", "img": "http://www.serebii.net/pokemongo/pokemon/087.png", "type": ["Water", "Ice"], "height": "1.70 m", "weight": "120.0 kg", "candy": "Seel Candy", "egg": "Not in Eggs", "spawn_chance": 0.013, "avg_spawns": 1.3, "spawn_time": "06:04", "multipliers": null, "weaknesses": ["Electric", "Grass", "Fighting", "Rock"], "prev_evolution": [{"num": "086", "name": "Seel"}]}',),
 ('{"id": 91, "num": "091", "name": "Cloyster", "img": "http://www.serebii.net/pokemongo/pokemon/091.png", "type": ["Water", "Ice"], "height": "1.50 m", "weight": "132.5 kg", "candy": "Shellder Candy", "egg": "Not in Eggs", "spawn_chance": 0.015, "avg_spawns": 1.5, "spawn_time": "02:33", "multipliers": null, "weaknesses": ["Electric", "Grass", "Fighting", "Rock"], "prev_evolution": [{"num": "090", "name": "Shellder"}]}',),
 ('{"id": 124, "num": "124", "name": "Jynx", "img": "http://www.serebii.net/pokemongo/pokemon/124.png", "type": ["Ice", "Psychic"], "height": "

same as above but with syntax sugar

In [19]:
db.query("""select body from docs where json_array_contains(body->>'$.type', 'Ice')""")

[('body',),
 ('{"id": 87, "num": "087", "name": "Dewgong", "img": "http://www.serebii.net/pokemongo/pokemon/087.png", "type": ["Water", "Ice"], "height": "1.70 m", "weight": "120.0 kg", "candy": "Seel Candy", "egg": "Not in Eggs", "spawn_chance": 0.013, "avg_spawns": 1.3, "spawn_time": "06:04", "multipliers": null, "weaknesses": ["Electric", "Grass", "Fighting", "Rock"], "prev_evolution": [{"num": "086", "name": "Seel"}]}',),
 ('{"id": 91, "num": "091", "name": "Cloyster", "img": "http://www.serebii.net/pokemongo/pokemon/091.png", "type": ["Water", "Ice"], "height": "1.50 m", "weight": "132.5 kg", "candy": "Shellder Candy", "egg": "Not in Eggs", "spawn_chance": 0.015, "avg_spawns": 1.5, "spawn_time": "02:33", "multipliers": null, "weaknesses": ["Electric", "Grass", "Fighting", "Rock"], "prev_evolution": [{"num": "090", "name": "Shellder"}]}',),
 ('{"id": 124, "num": "124", "name": "Jynx", "img": "http://www.serebii.net/pokemongo/pokemon/124.png", "type": ["Ice", "Psychic"], "height": "

In [20]:
db.close()